In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [2]:
!pip install -q tensorflow-hub

In [4]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [2]:
#load dataset
data = pd.read_csv('processed.cleveland.data', header=None)
data.columns
print(data.head())

     0    1    2      3      4    5    6      7    8    9    10   11   12  13
0  63.0  1.0  1.0  145.0  233.0  1.0  2.0  150.0  0.0  2.3  3.0  0.0  6.0   0
1  67.0  1.0  4.0  160.0  286.0  0.0  2.0  108.0  1.0  1.5  2.0  3.0  3.0   2
2  67.0  1.0  4.0  120.0  229.0  0.0  2.0  129.0  1.0  2.6  2.0  2.0  7.0   1
3  37.0  1.0  3.0  130.0  250.0  0.0  0.0  187.0  0.0  3.5  3.0  0.0  3.0   0
4  41.0  0.0  2.0  130.0  204.0  0.0  2.0  172.0  0.0  1.4  1.0  0.0  3.0   0


In [44]:
#define target and variable 
X = data.drop(13, axis=1)
y = data[13]

In [52]:
# Separate features and target
X = data.iloc[:, :-1]
y = data.iloc[:, -1]

In [56]:
continuous_cols= X.iloc[:, [0, 3, 4, 7, 9, 11]]
categorical_cols = X.iloc[:, [1, 2, 5, 6, 8, 10, 12]]
print(continuous_cols)
print(categorical_cols)

       0      3      4      7    9    11
0    63.0  145.0  233.0  150.0  2.3  0.0
1    67.0  160.0  286.0  108.0  1.5  3.0
2    67.0  120.0  229.0  129.0  2.6  2.0
3    37.0  130.0  250.0  187.0  3.5  0.0
4    41.0  130.0  204.0  172.0  1.4  0.0
..    ...    ...    ...    ...  ...  ...
298  45.0  110.0  264.0  132.0  1.2  0.0
299  68.0  144.0  193.0  141.0  3.4  2.0
300  57.0  130.0  131.0  115.0  1.2  1.0
301  57.0  130.0  236.0  174.0  0.0  1.0
302  38.0  138.0  175.0  173.0  0.0  NaN

[303 rows x 6 columns]
      1    2    5    6    8    10   12
0    1.0  1.0  1.0  2.0  0.0  3.0  6.0
1    1.0  4.0  0.0  2.0  1.0  2.0  3.0
2    1.0  4.0  0.0  2.0  1.0  2.0  7.0
3    1.0  3.0  0.0  0.0  0.0  3.0  3.0
4    0.0  2.0  0.0  2.0  0.0  1.0  3.0
..   ...  ...  ...  ...  ...  ...  ...
298  1.0  1.0  0.0  0.0  0.0  2.0  7.0
299  1.0  4.0  1.0  0.0  0.0  2.0  7.0
300  1.0  4.0  0.0  0.0  1.0  2.0  7.0
301  0.0  2.0  0.0  2.0  0.0  2.0  3.0
302  1.0  3.0  0.0  0.0  0.0  1.0  3.0

[303 rows x 7 c

In [55]:
X.replace('?', np.nan, inplace=True)

In [57]:
# Imputers : mean and mode
imp_cat = SimpleImputer(strategy='most_frequent')
imp_cont = SimpleImputer(strategy='mean')

In [ ]:
#impute missing values
X_cont_imputed = pd.DataFrame(imp_cont.fit_transform(continuous_cols), columns=continuous_cols.columns)
X_cat_imputed = pd.DataFrame(imp_cat.fit_transform(categorical_cols), columns=categorical_cols.columns)


1     False
2     False
5     False
6     False
8     False
10    False
12    False
dtype: bool

In [75]:
#one hot encode categorical variable    
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(drop='first')
ohe_cat = encoder.fit_transform(X_cat_imputed).toarray()
print(ohe_cat)

[[1. 0. 0. ... 1. 1. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 1.]
 ...
 [1. 0. 0. ... 0. 0. 1.]
 [0. 1. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]]


In [76]:
#scale continuous variable
scaler = StandardScaler()
X_cont_scaled = scaler.fit_transform(X_cont_imputed)
print(X_cont_scaled)

[[ 0.94872647  0.75752504 -0.2649003   0.01719733  1.08733806 -0.72309499]
 [ 1.39200191  1.61121989  0.76041519 -1.82190531  0.39718162  2.50385129]
 [ 1.39200191 -0.6652997  -0.34228261 -0.90235399  1.34614673  1.42820253]
 ...
 [ 0.28381332 -0.0961698  -2.23814899 -1.51538821  0.13837295  0.35255377]
 [ 0.28381332 -0.0961698  -0.20686358  1.06811312 -0.89686172  0.35255377]
 [-1.82174501  0.35913411 -1.38694368  1.02432497 -0.89686172  0.        ]]


In [79]:
#ohe target variable
nominal_column = y
enc = OneHotEncoder(categories='auto')
df_hd_named_enc = pd.DataFrame(enc.fit_transform(
    y).toarray())
df_ohe = df_hd_named_enc.columns = enc.get_feature_names_out(y)
df_ohe.head(10)

ValueError: Expected a 2-dimensional container but got <class 'pandas.core.series.Series'> instead. Pass a DataFrame containing a single row (i.e. single sample) or a single column (i.e. single feature) instead.